![Alt text](image1.png)


![Alt text](image2.png)


![Alt text](image3.png)


In [1]:
# Preparing 
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

df = pd.read_csv("DatasetExos.csv",nrows=5000, sep=',')
df.head()


,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Exercise,Category,Set
0,2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,30.0
1,2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,30.0
2,2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,30.0
3,2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,30.0
4,2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,30.0


on remarque que les colonnes 1 8 9 10 ne sont pas numerique donc on les supprime

In [2]:
df_new=df.drop(df.columns[[0, 7, 8, 9]], axis=1)
df_new.head()
print(df_new)

         Acc_x     Acc_y     Acc_z   Gyro_x   Gyro_y  Gyro_z   Set
0     0.013500  0.977000 -0.071000  -1.8904   2.4392  0.9388  30.0
1    -0.001500  0.970500 -0.079500  -1.6826  -0.8904  2.1708  30.0
2     0.001333  0.971667 -0.064333   2.5608  -0.2560 -1.4146  30.0
3    -0.024000  0.957000 -0.073500   8.0610  -4.5244 -2.0730  30.0
4    -0.028000  0.957667 -0.115000   2.4390  -1.5486 -3.6098  30.0
...        ...       ...       ...      ...      ...     ...   ...
4995 -0.045500 -1.074500 -0.072000  15.8534  -0.3782 -6.8172   6.0
4996 -0.004000 -0.911667  0.002667  57.9756   1.3536 -1.9268   6.0
4997  0.024500 -0.698500  0.175500  70.9880 -13.6952  4.4148   6.0
4998  0.015000 -0.855333  0.401000   9.2194  -3.0730  2.7682   6.0
4999  0.009000 -0.916000  0.465500 -28.5364   3.8780  7.3778   6.0

[5000 rows x 7 columns]


In [3]:
print(df_new.applymap(lambda x: isinstance(x, (int, float))))#verification des valeurs si reel ou entiere
#on remarque que les valeur ne sont ni ree ni entier à cause du .

      Acc_x  Acc_y  Acc_z  Gyro_x  Gyro_y  Gyro_z   Set
0      True   True   True    True    True    True  True
1      True   True   True    True    True    True  True
2      True   True   True    True    True    True  True
3      True   True   True    True    True    True  True
4      True   True   True    True    True    True  True
...     ...    ...    ...     ...     ...     ...   ...
4995   True   True   True    True    True    True  True
4996   True   True   True    True    True    True  True
4997   True   True   True    True    True    True  True
4998   True   True   True    True    True    True  True
4999   True   True   True    True    True    True  True

[5000 rows x 7 columns]


/var/folders/x0/9l9lt4g56dl5nws_qscbtj6r0000gn/T/ipykernel_1070/1299363248.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df_new.applymap(lambda x: isinstance(x, (int, float))))#verification des valeurs si reel ou entiere


In [4]:
#on converti les point en virgule
def convertir_virgule_en_point(val):
    str_val=str(val).replace(',', '.').replace('.', '', str(val).count('.') - 1)
    
    #convertion en float
    try:
        float_value=float(str_val) if str_val else None
    except ValueError:
        float_value=None  
    return float_value


df_new=df_new.applymap(convertir_virgule_en_point)

#virification de la convertion
print(df_new.applymap(lambda x: isinstance(x, (int, float))))

      Acc_x  Acc_y  Acc_z  Gyro_x  Gyro_y  Gyro_z   Set
0      True   True   True    True    True    True  True
1      True   True   True    True    True    True  True
2      True   True   True    True    True    True  True
3      True   True   True    True    True    True  True
4      True   True   True    True    True    True  True
...     ...    ...    ...     ...     ...     ...   ...
4995   True   True   True    True    True    True  True
4996   True   True   True    True    True    True  True
4997   True   True   True    True    True    True  True
4998   True   True   True    True    True    True  True
4999   True   True   True    True    True    True  True

[5000 rows x 7 columns]


/var/folders/x0/9l9lt4g56dl5nws_qscbtj6r0000gn/T/ipykernel_1070/190011261.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_new=df_new.applymap(convertir_virgule_en_point)
/var/folders/x0/9l9lt4g56dl5nws_qscbtj6r0000gn/T/ipykernel_1070/190011261.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df_new.applymap(lambda x: isinstance(x, (int, float))))


In [5]:
#methode2
def discretiser_intervalle_egales(data):
    #calculer le nombre d interval k en utilisant la formule  donne au tp
    k=round(1 + (3/10) * np.log10(len(data)))

    #calculer les valeur minimale et maximale du dataset
    min_val=np.min(data)
    max_val=np.max(data)

    #calcule de la largeur de chaque interval
    interval_width=(max_val - min_val) / k

    #creation des interval 
    intervals = [(min_val + i * interval_width, min_val + (i + 1) * interval_width) for i in range(k)]

    #discretisation
    discretized_data = np.digitize(data, [interval[1] for interval in intervals])

    return discretized_data

In [6]:
#fonction qui calcule moyenne interval
def calculer_moyennes_intervalles(data, donnees_discretes):
    etiquettes_uniques=np.unique(donnees_discretes)
    moyennes_intervalles=[]

    for etiquette in etiquettes_uniques:
        donnees_intervalle=data[donnees_discretes == etiquette]
        moyenne_intervalle=np.mean(donnees_intervalle)
        moyennes_intervalles.append(moyenne_intervalle)

    return np.array(moyennes_intervalles)

In [7]:
def extract_columns_types(df):
    # columns=list(df)
    
    types= df.dtypes
    return types
types=extract_columns_types(df_new)
print("les types des colonnes du dataset sont", types)

les types des colonnes du dataset sont Acc_x     float64
Acc_y     float64
Acc_z     float64
Gyro_x    float64
Gyro_y    float64
Gyro_z    float64
Set       float64
dtype: object


In [8]:
#normalisation min-max
def normalise_value(data, new_min=0, new_max=1):
    
    # Forcer la conversion en type float pour assurer des données numériques
    #data = pd.to_numeric(data, errors='coerce')
    min_val = np.min(data)
    max_val = np.max(data)
    if min_val == max_val:
        #dans le cas ou min=max on remplace le reste par le min pour eviter la division par 0
        return np.full(data.shape, new_min)
    else:
        #la formule
        normalized_data = ((data - min_val) / (max_val - min_val)) * (new_max - new_min) + new_min
        return normalized_data


In [9]:
k = round(1 + (3/10) * np.log10(len(df_new)))
print(k)

2


In [10]:
donnees_discretes=discretiser_intervalle_egales(df_new)
#print(donnees_discretes)
df_discretise=pd.DataFrame(donnees_discretes)

df_discretise.head()
print(df_discretise)

      0  1  2  3  4  5  6
0     1  1  1  1  1  1  1
1     1  1  1  1  1  1  1
2     1  1  1  1  1  1  1
3     1  1  1  1  1  1  1
4     1  1  1  1  1  1  1
...  .. .. .. .. .. .. ..
4995  1  1  1  1  1  1  1
4996  1  1  1  1  1  1  1
4997  1  1  1  1  0  1  1
4998  1  1  1  1  1  1  1
4999  1  1  1  0  1  1  1

[5000 rows x 7 columns]


In [11]:
moyennes_intervalles=calculer_moyennes_intervalles(df_new,donnees_discretes)
moyennes_intervalles

array([5.04808873, 7.7029946 , 9.69878289])

In [12]:
df_normalized = df_new.copy()
for col in df_new.columns:
    df_normalized[col] = normalise_value(df_new[col], new_min=0, new_max=1)

print("DataFrame après normalisation :")
print(df_normalized.head())

DataFrame après normalisation :
      Acc_x     Acc_y     Acc_z    Gyro_x    Gyro_y    Gyro_z       Set
0  0.999934  0.791869  0.328671  0.440627  0.514566  0.311741  0.292135
1  0.999932  0.789829  0.323576  0.441746  0.494928  0.311827  0.292135
2  0.999932  0.790195  0.332667  0.464599  0.498669  0.311577  0.292135
3  0.999930  0.785591  0.327173  0.494220  0.473494  0.311532  0.292135
4  0.999929  0.785800  0.302298  0.463943  0.491046  0.311424  0.292135
